Connected to downstream (Python 3.11.11)

In [ ]:
import os
import scanpy as sc
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sb
from matplotlib import colors
from matplotlib import rcParams
import seaborn as sns
from gprofiler import GProfiler
import sys
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
import configparser

# Read configuration file
config = configparser.ConfigParser()
config.read("../../utils/config.ini")

rawPath = config.get("DEFAULT", "rawPath")
scriptsPath = config.get("DEFAULT", "scriptsPath")
figPath = config.get("DEFAULT", "figPath")
gprofFolder = scriptsPath + "9_testing/GProfiler/"

integrationDir = rawPath + 'integration/metacells/cancer/'

path = "/group/testa/Project/OvarianAtlas/atlas_project/raw_data/out_of_sample_extension_backup/out_of_sample_extension/fibroblasts/integrated_query_zheng_seacells_scarches_tissuetreat_predicted_cellstates.h5ad"
adata = sc.read_h5ad(path)

In [ ]:
marta_before_integration = sc.read_h5ad("/group/testa/Project/OvarianAtlas/atlas_project/raw_data/metacells_backup/metacells/fibroblasts/seacells_hdg_patients.h5ad")

In [ ]:
marta_before_integration

AnnData object with n_obs × n_vars = 6639 × 17063
    obs: 'ID', 'sample_name', 'tissue', 'developmental_stage', 'treatment', 'recurrence', 'tumor_stage', 'paper_ID', 'anatomical_location', 'patient_id', 'dataset', 'cell_type', 'cell_subtype', 'sample_ID', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'CancerMSK', 'EndothelialMSK', 'FibroblastsMSK', 'HematopoieticMSK', 'cell_labels_ratio', 'max', 'assignment', 'leiden-1.8', 'SEACell', '# Single Cells'
    layers: 'raw'

In [ ]:
marta_after_integration = sc.read_h5ad("/group/testa/Project/OvarianAtlas/atlas_project/raw_data/integration_backup/integration/metacells/fibroblasts/seacells_hdg_patients_batch_corr_scgen_celltypes_HDG.h5ad")

In [ ]:
marta_after_integration

AnnData object with n_obs × n_vars = 6639 × 5179
    obs: 'ID', 'sample_name', 'tissue', 'developmental_stage', 'treatment', 'recurrence', 'tumor_stage', 'paper_ID', 'anatomical_location', 'patient_id', 'dataset', 'cell_type', 'cell_subtype', 'sample_ID', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'CancerMSK', 'EndothelialMSK', 'FibroblastsMSK', 'HematopoieticMSK', 'cell_labels_ratio', 'max', 'assignment', 'leiden-1.8', 'SEACell', '# Single Cells', 'tissue-treatment', 'iCAF', 'mCAF', 'vCAF', 'starCAF', 'cell_types', '_scvi_batch', '_scvi_labels'
    uns: '_scvi_manager_uuid', '_scvi_uuid'
    obsm: 'corrected_latent', 'latent'

In [ ]:
marta_after_integration_raw = marta_after_integration.raw.to_adata()

In [ ]:
marta_after_integration_raw

AnnData object with n_obs × n_vars = 6639 × 17063
    obs: 'ID', 'sample_name', 'tissue', 'developmental_stage', 'treatment', 'recurrence', 'tumor_stage', 'paper_ID', 'anatomical_location', 'patient_id', 'dataset', 'cell_type', 'cell_subtype', 'sample_ID', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'CancerMSK', 'EndothelialMSK', 'FibroblastsMSK', 'HematopoieticMSK', 'cell_labels_ratio', 'max', 'assignment', 'leiden-1.8', 'SEACell', '# Single Cells', 'tissue-treatment', 'iCAF', 'mCAF', 'vCAF', 'starCAF', 'cell_types', '_scvi_batch', '_scvi_labels'
    var: 'highly_variable'
    uns: '_scvi_manager_uuid', '_scvi_uuid'
    obsm: 'corrected_latent', 'latent'

In [ ]:
marta_before_integration.obs_names == marta_after_integration_raw.obs_names

array([ True,  True,  True, ...,  True,  True,  True])

In [ ]:
all(marta_before_integration.obs_names == marta_after_integration_raw.obs_names)

True

In [ ]:
all(marta_before_integration.var_names == marta_after_integration_raw.var_names)

True

In [ ]:
from scipy.sparse import csr_matrix
import numpy as np

# Assuming marta_before_integration and marta_after_integration_raw are already defined
# and their .X attributes are csr_matrix objects

def compare_csr_matrices(matrix1: csr_matrix, matrix2: csr_matrix) -> bool:
    """
    Compare two csr_matrix objects for equality.
    
    Parameters:
    matrix1 (csr_matrix): First sparse matrix.
    matrix2 (csr_matrix): Second sparse matrix.
    
    Returns:
    bool: True if the matrices are equal, False otherwise.
    """
    if matrix1.shape != matrix2.shape:
        return False
    
    return (matrix1 != matrix2).nnz == 0

# Example usage
are_equal = compare_csr_matrices(marta_before_integration.X, marta_after_integration_raw.X)


In [ ]:
are_equal

False

In [ ]:
marta_before_integration

AnnData object with n_obs × n_vars = 6639 × 17063
    obs: 'ID', 'sample_name', 'tissue', 'developmental_stage', 'treatment', 'recurrence', 'tumor_stage', 'paper_ID', 'anatomical_location', 'patient_id', 'dataset', 'cell_type', 'cell_subtype', 'sample_ID', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'CancerMSK', 'EndothelialMSK', 'FibroblastsMSK', 'HematopoieticMSK', 'cell_labels_ratio', 'max', 'assignment', 'leiden-1.8', 'SEACell', '# Single Cells'
    layers: 'raw'

In [ ]:
marta_before_integration.X

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 55854324 stored elements and shape (6639, 17063)>

In [ ]:
marta_before_integration

AnnData object with n_obs × n_vars = 6639 × 17063
    obs: 'ID', 'sample_name', 'tissue', 'developmental_stage', 'treatment', 'recurrence', 'tumor_stage', 'paper_ID', 'anatomical_location', 'patient_id', 'dataset', 'cell_type', 'cell_subtype', 'sample_ID', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'CancerMSK', 'EndothelialMSK', 'FibroblastsMSK', 'HematopoieticMSK', 'cell_labels_ratio', 'max', 'assignment', 'leiden-1.8', 'SEACell', '# Single Cells'
    layers: 'raw'

In [ ]:
sc.get.obs_df(marta_before_integration, "MT-C03")

KeyError: "Could not find keys '['MT-C03']' in columns of `adata.obs` or in adata.var_names."

In [ ]:
sc.get.var_df(marta_before_integration, "MT-C03")

KeyError: "Could not find keys '['MT-C03']' in columns of `adata.var` or in adata.obs_names."

In [ ]:
sc.get.obs_df(marta_before_integration, "MT-CO3")

,MT-CO3
SEACell_patient_tissue,
SEACell-137_Geistlinger_59_Metastasis,245.781372
SEACell-76_Geistlinger_59_Metastasis,673.179871
SEACell-108_Geistlinger_59_Metastasis,725.450073
SEACell-135_Geistlinger_59_Metastasis,209.973969
SEACell-101_Geistlinger_59_Metastasis,368.108002
...,...
SEACell-9_Vasquez_118_Metastasis,134.788818
SEACell-49_Vasquez_118_Metastasis,74.147285
SEACell-21_Vasquez_118_Metastasis,41.178391


In [ ]:
sc.get.obs_df(marta_after_integration_raw, "MT-CO3")

,MT-CO3
SEACell_patient_tissue,
SEACell-137_Geistlinger_59_Metastasis,5.508503
SEACell-76_Geistlinger_59_Metastasis,6.513497
SEACell-108_Geistlinger_59_Metastasis,6.588170
SEACell-135_Geistlinger_59_Metastasis,5.351735
SEACell-101_Geistlinger_59_Metastasis,5.911089
...,...
SEACell-9_Vasquez_118_Metastasis,4.911101
SEACell-49_Vasquez_118_Metastasis,4.319450
SEACell-21_Vasquez_118_Metastasis,3.741908


No kernel connected